# Parameters

In [1]:
pwd

'/home/whirldata/Documents/Pytorch/notebooks/BERT'

In [37]:
source_folder = '/home/whirldata/Documents/Pytorch/notebooks/BERT'
destination_folder = '/home/whirldata/Documents/Pytorch/notebooks/BERT/saved_models'

In [38]:
#Company Name	Business Description	Industry Classification Tag


# Libraries

In [39]:
!pip install transformers

In [40]:
!pip install torchtext

In [41]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import f1_score,accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [42]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Preliminaries

In [43]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
full_data=pd.read_excel('/home/whirldata/Documents/Pytorch/notebooks/BERT/Train.xlsx')

In [ ]:
full_data

In [ ]:
len(full_data)

In [ ]:
train_df=full_data[0:5043]

In [ ]:
valid_df=full_data[5044:-1]

In [ ]:
len(train_df)

In [ ]:
len(valid_df)

In [ ]:
train_df.to_csv("train_data.csv")

In [ ]:
valid_df.to_csv("valid_data.csv")

In [ ]:
full_data_test=pd.read_excel('/home/whirldata/Documents/Pytorch/notebooks/BERT/Test.xlsx')

In [ ]:
full_data_test.to_csv("test_data.csv")

In [ ]:
ls

In [44]:
# Model parameter
MAX_SEQ_LEN = 120
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('Company_Name', text_field), ('Business_Description', text_field), ('Industry_Classification_Tag', label_field)]

#Company Name	Business Description	Industry Classification Tag

# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train_data.csv', validation='valid_data.csv', test='test_data.csv',
                                           format='CSV', fields=fields, skip_header=True)

# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.Company_Name),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.Company_Name),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=16, device=device, train=False, shuffle=False, sort=False)

# Models

In [45]:
class BERT_Single(nn.Module):

    def __init__(self):
        super(BERT_Single, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [46]:
class OurClassifier(nn.Module):

    def __init__(self, dimension):
        super(OurClassifier, self).__init__()

        self.fc = nn.Linear(dimension, 1)

    def forward(self, x):
        x = self.fc(x)
        x = torch.squeeze(x, 1)
        x = torch.sigmoid(x)

        return x

In [47]:
class BERT_Double(nn.Module):

    def __init__(self):
        super(BERT_Double, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder1 = BertForSequenceClassification.from_pretrained(options_name)
        self.encoder2 = BertForSequenceClassification.from_pretrained(options_name)
        self.classifier = OurClassifier(6)

    def forward(self, title, text, label):
        title_loss, title_fea = self.encoder1(title, labels=label)[:2]
        text_loss, text_fea = self.encoder2(text, labels=label)[:2]

        total_fea = torch.cat((title_fea, text_fea), 1)
        total_out = self.classifier(total_fea)

        return total_out

In [48]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

# Training

In [49]:
# Save and Load Functions

def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [50]:
# Training Function

def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 20,
          eval_every = len(train_iter) // 2,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (Company_Name,Business_Description,Industry_Classification_Tag), _ in train_loader:
            
            Company_Name = Company_Name.type(torch.LongTensor)
            Company_Name =Company_Name.to(device)
            Business_Description =Business_Description.type(torch.LongTensor)
            Business_Description =Business_Description.to(device)
            Industry_Classification_Tag = Industry_Classification_Tag.type(torch.LongTensor)
            Industry_Classification_Tag =Industry_Classification_Tag.to(device)
#             labels = labels.type(torch.LongTensor)           
#             labels = labels.to(device)
#             titletext = titletext.type(torch.LongTensor)  
#             titletext = titletext.to(device)
#             output = model(titletext, labels)
            output=model(Company_Name,Business_Description,Industry_Classification_Tag)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    

                    # validation loop
                    for (Company_Name,Business_Description,Industry_Classification_Tag), _ in valid_loader:
                        Company_Name = Company_Name.type(torch.LongTensor)
                        Company_Name =Company_Name.to(device)
                        Business_Description =Business_Description.type(torch.LongTensor)
                        Business_Description =Business_Description.to(device)
                        Industry_Classification_Tag = Industry_Classification_Tag.type(torch.LongTensor)
                        Industry_Classification_Tag =Industry_Classification_Tag.to(device)
                        output=model(Company_Name,Business_Description,Industry_Classification_Tag)
                        loss, _ = output
#                         labels = labels.type(torch.LongTensor)           
#                         labels = labels.to(device)
#                         titletext = titletext.type(torch.LongTensor)  
#                         titletext = titletext.to(device)
#                         output = model(titletext, labels)
#                         loss, _ = output
                        
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')
#

In [1]:
model = BERT()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

train(model=model, optimizer=optimizer)

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/title_metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show() 